In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [3]:
train, test = pd.read_csv('data/train.csv'), pd.read_csv('data/test.csv')